In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import os

In [3]:
connect_info = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format('root','password','localhost',3306,'settlement_test')  #1
engine = create_engine(connect_info)

# 確認清單

In [78]:
def afterSettleList(session,method='append'):
#     check_exist=pd.read_sql('select * from afterSettleList where session='+str(session),con=engine)#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:    
    df=pd.read_sql('select * from confirmPurchaseRight where session='+str(session),con=engine,index_col=['session','fbuid'])

    dfx=df.loc[:,['username','getSure','wineID','wineName','limit','orginalOrderCount','orderCount','giftBottle','orderCountAfterSale',
                 'price','totalPrice']]
    dfx['limit']=dfx['limit'].apply(lambda s:"限加"+str(s) if s>0 else '無限加限制')
    df_sorry=dfx[dfx['getSure']=='sorry']
    df_sorry['orderCountAfterSale']=0
    df_sorry['totalPrice']=0

    df_get=dfx[dfx['getSure']=='get']

    final_list=pd.concat([df_get,df_sorry])
    final_list=final_list.rename(columns={'username':'FB姓名','getSure':'購買權確認','wineID':'商品編號','wineName':'商品名稱',
                          'limit':'購買限制','orginalOrderCount':'留言購買數','orderCount':'結算確認購買數量','giftBottle':'優惠贈送量',
                         'orderCountAfterSale':'扣除贈送量後計價數','price':'商品單價','totalPrice':'商品總價'})
    final_list['備註']=['未排到' if (a=='sorry') else '限購品項,若留言購買數大於限夠數,自動修正。' if (a=='get')&(b!='無限加限制')&(c>0)
                      else '留言購買數大於庫存數,修正為最大可買數。' if (a=='get')&(b=='無限加限制')&(c>d) else '已確認'
                      for a,b,c,d in zip(final_list['購買權確認'],final_list['購買限制'],final_list['留言購買數'],final_list['結算確認購買數量'])]
    final_list=final_list.reset_index()
    final_list.to_sql(name='afterSettleList', con=engine, if_exists=method)
#     else:
#         print('afterSettleList session is duplicated')
    return final_list.sort_index()

afterSettleList('201908')


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,session,fbuid,FB姓名,購買權確認,商品編號,商品名稱,購買限制,留言購買數,結算確認購買數量,優惠贈送量,扣除贈送量後計價數,商品單價,商品總價,備註
0,201908,688753070,Shen Lin,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
1,201908,1133912470,Chialin Liao,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
2,201908,100000332146403,Yi-Hung Pan,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
3,201908,592053470,Jessica LuLu,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
4,201908,100007536616399,鄭建中,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
5,201908,573504806,Alex Kao,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
6,201908,100001491233237,Don Wu,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
7,201908,1331272331,Simon Chuang,get,3053,Caiarossa 2010 IGT Tuscany 紅酒,無限加限制,1,1,0,1,1680,1680,已確認
8,201908,100002501697401,潘宥瑞,get,4344,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,無限加限制,1,1,0,1,490,490,已確認
9,201908,100000128365914,Chung Yu Chen,get,4344,Stephane Aviron 2016 “Moulin a Vent” Vieilles ...,無限加限制,1,1,0,1,490,490,已確認


In [4]:
def feeList(session,method='append'):
#     check_exist=pd.read_sql('select * from feeList where session='+str(session),con=engine)#防止重複結算
#     check_exist['session']=check_exist['session'].astype(str)
#     if session not in check_exist['session'].values:
    df=pd.read_sql('select * from calDiliverCostFinal where session='+str(session),con=engine,index_col=['session','fbuid'])
    #製作訂單編號
    df['index']=(df['index']+1).apply(lambda s:'000'+str(s) if s<10 else '00'+str(s) if s<100  else '0'+str(s) if s<1000 else str(s))
    df['訂單編號']=str(session)+df['index']
    df=df.loc[:,['訂單編號','orderCount','deilverFreeAll','deilverFreeAllbottle','discountAll','selfGetCost','diliverCostAfterDiscountAll',
                'diliverCostAfterDeilverFreeAll','totalPriceIncludeDC']]

    df['deilverFreeAll']=df['deilverFreeAll'].apply(lambda s:"滿"+str(s)+'元免運' if s>0 else '無')
    df['deilverFreeAllbottle']=df['deilverFreeAllbottle'].apply(lambda s:"滿"+str(s)+'件免運' if s>0 else '無')
    df['discountAll']=df['discountAll'].apply(lambda s:"全團"+str(s*10)+'折優惠' if s!=1 else '無全團折扣優惠')

    df=df.rename(columns={'orderCount':'結算確認購買數量','deilverFreeAll':'金額達標免運優惠','deilverFreeAllbottle':'瓶數達標免運優惠',
                          'discountAll':'全團折扣優惠','selfGetCost':'折扣前價格','diliverCostAfterDiscountAll':'折扣後自取價',
                          'diliverCostAfterDeilverFreeAll':'運費','totalPriceIncludeDC':'含運費總價'})
    df=df.reset_index()
#     df.to_sql(name='feeList', con=engine, if_exists=method)
#     else:
#         print('feeList session is duplicated')
    return df.sort_index()

feeList('201908')

/Users/ankycheng/Google_Drive/Projects/1 Working/Innopro FB Crawler/Innopro/venv/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,session,fbuid,訂單編號,結算確認購買數量,金額達標免運優惠,瓶數達標免運優惠,全團折扣優惠,折扣前價格,折扣後自取價,運費,含運費總價
0,201908,36914827,2019080001,4,無,滿6件免運,無全團折扣優惠,4700,4700,225,4925
1,201908,501609342,2019080002,8,無,滿6件免運,無全團折扣優惠,6580,6580,0,6580
2,201908,507948606,2019080003,2,無,滿6件免運,無全團折扣優惠,13000,13000,225,13225
3,201908,513030038,2019080004,3,無,滿6件免運,無全團折扣優惠,1880,1880,225,2105
4,201908,518125422,2019080005,4,無,滿6件免運,無全團折扣優惠,3260,3260,225,3485
5,201908,519998038,2019080006,1,無,滿6件免運,無全團折扣優惠,300,300,160,460
6,201908,524402757,2019080007,3,無,滿6件免運,無全團折扣優惠,3660,3660,225,3885
7,201908,525152873,2019080008,2,無,滿6件免運,無全團折扣優惠,1290,1290,225,1515
8,201908,526487238,2019080009,1,無,滿6件免運,無全團折扣優惠,7710,7710,160,7870
9,201908,526497993,2019080010,1,無,滿6件免運,無全團折扣優惠,790,790,160,950
